# Logistic regression

Although having "regression" in its name, logistic regression is a classification (rather than regression) algorithm. Instead of modelling the qualitative response $y$ directly, logistic regression models the probability that $y$ belongs to a particular class (category).

Watch the 8-minute video below for a visual explanation of logistic regression:

```{admonition} Video

<iframe width="700" height="394" src="https://www.youtube.com/embed/yIYKR4sgzI8?start=24" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

[Explaining Logistic Regression by StatQuest](https://www.youtube.com/embed/yIYKR4sgzI8?start=24), embedded according to [YouTube's Terms of Service](https://www.youtube.com/static?gl=CA&template=terms).

```

## Import libraries and load data

Get ready by importing the APIs needed from respective libraries.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, RocCurveDisplay

from statsmodels.formula.api import logit

%matplotlib inline

Load the [Default dataset](https://github.com/pykale/transparentML/blob/main/data/Default.csv) and inspect the first three rows.

In [ ]:
data_url = "https://github.com/pykale/transparentML/raw/main/data/Default.csv"
df = pd.read_csv(data_url)

# Note: factorize() returns two objects: a label array and an array with the unique values. We are only interested in the first object.
df["default2"] = df.default.factorize()[0]
df["student2"] = df.student.factorize()[0]
df.head(3)

## Logistic model

Logistic regression models the probability that the response $y$ belongs to a particular class (category) using linear _regression_ via a _logistic_ function, hence the name _logistic regression_. For binary classification, let us use 1 to denote the positive class (e.g. "Yes") and 0 to denote the negative class (e.g. "No").

### Logit function transformation

Let $\pi$ be the probability of a positive outcome 

$$
\pi = \mathbb{P}(y=1| x).
$$

Then the probability of a negative outcome $ \mathbb{P}(y=0| x) = 1-\pi $. 

The [odds](https://en.wikipedia.org/wiki/Odds) of a positive outcome is the ratio of the probability of a positive outcome to the probability of a negative outcome, i.e.

$$
\frac{\pi}{1-\pi}.
$$

The log of the odds is called the [logit](https://en.wikipedia.org/wiki/Logit) and the logit function is defined as :

$$
\mathrm{logit}(\pi)=\log \frac{\pi}{1-\pi}
$$ 

The logit function is a monotonic transformation of the probability $\pi$. It maps the probability $\pi$ ranging from 0 to 1 to the real line ranging from $-\infty$ to $\infty$. The logit function is also known as the log-odds function.

### Logistic function

In simple linear regression, we have modelled the relationship between outcome $y$ and feature $x$ with a linear equation:

$$
y = \beta_0 + \beta_1 x.
$$

This linear regression model is not able to produce a binary outcome for binary classification or a probability (in the range from 0 to 1 only). With the logit function above, we can transform the probability $\pi$ ranging from 0 to 1 to the logit function $\mathrm{logit}(\pi)$ ranging from $-\infty$ to $\infty$. Now we can use the linear regression model to model the logit function $\mathrm{logit}(\pi)$ instead of the probability $\pi$ or $y$ directly. The logit function is defined as:

$$
\mathrm{logit}(\pi)=\log \frac{\pi}{1-\pi} = \beta_0 + \beta_1 x.
$$

The [logistic function](https://en.wikipedia.org/wiki/Logistic_function), also known as the sigmoid function, is the inverse of the logit function. It maps the logit function $\mathrm{logit}(\pi)$ ranging from $-\infty$ to $\infty$ to the probability $\pi$ ranging from 0 to 1:

$$
\pi = \mathbb{P}(y = 1 | x) = \mathrm{logit}^{-1}(\beta_0 + \beta_1 x) = \mathrm{logistic}(\beta_0 + \beta_1 x) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x_1)}}.
$$

<!-- For classification, we cannot use linear regression to model $y$ directly, because $y$ is either. Instead, we can model the logit of the probability of a positive outcome:

 we prefer probabilities between 0 and 1, so we wrap the right side of the equation into the logistic function. This forces the output to assume only values between 0 and 1 -->


<!-- Specifically, instead of fitting a straight line or hyperplane, the logistic regression model uses the logistic function to squeeze the output of a linear equation between 0 and 1. 

$$
\textrm{logistic}(x) = \frac{1}{1 + e^{-x}}
$$ -->

## Estimating the coefficients to make predictions

The coefficients of a logistic regression model can be estimated by [maximum likelihood estimation](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation) (MLE). The likelihood function is

$$
\mathcal{L}(\beta_0, \beta_1) = \prod_{i:y_i=1} \mathbb{P}(y_i = 1) \prod_{i:y_i=0} (1 - \mathbb{P}(y_i = 1)).
$$

The mathematical details are beyond the scope of this course. If interested, please read Section 4.3 of the [Pattern Recognition and Machine Learning](https://www.microsoft.com/en-us/research/publication/pattern-recognition-machine-learning/) book for more details of the optimization for logistic regression.

To make predictions, taking the classification problem on the [Default dataset](https://github.com/pykale/transparentML/blob/main/data/Default.csv), the probability of default given balance predicted by a logistic regression model is

$$
\mathbb{P}(\text{default} = \text{Yes} \mid \text{balance}, \beta_0, \beta_1) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 \times \text{balance})}}.
$$

### Example explanation of system transparency

Let us to see the curve of the logistic function on the [Default dataset](https://github.com/pykale/transparentML/blob/main/data/Default.csv).

In [ ]:
sns.regplot(data=df, x="balance", y="default2", logistic=True)
plt.ylabel("Probability of default")
plt.xlabel("Balance")
plt.show()

We can see an S-shaped curve with the vertical axis ranging from 0 to 1, indicating the probability of a positive outcome (`default`) with respect to the horizontal axis $x$ (`balance`). 

In practice, we can use the `scikit-learn` or `statsmodels` package to fit a logistic regression model and make predictions. 

Let us fit a logistic regression model using the [`scikit-learn` API for logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) first.

In [ ]:
clf = LogisticRegression(solver="newton-cg")
X_train = df.balance.values.reshape(-1, 1)
X_test = np.arange(df.balance.min(), df.balance.max()).reshape(-1, 1)
y = df.default2
clf.fit(X_train, y)
print(clf)
print("classes: ", clf.classes_)
print("coefficients: ", clf.coef_)
print("intercept :", clf.intercept_)

Let us plot the fitted logistic regression model and study the system transparency.

In [ ]:
prob = clf.predict_proba(X_test)

plt.scatter(X_train, y, color="orange")
plt.plot(X_test, prob[:, 1], color="lightblue")

plt.hlines(
    1,
    xmin=plt.gca().xaxis.get_data_interval()[0],
    xmax=plt.gca().xaxis.get_data_interval()[1],
    linestyles="dashed",
    lw=1,
)
plt.hlines(
    0,
    xmin=plt.gca().xaxis.get_data_interval()[0],
    xmax=plt.gca().xaxis.get_data_interval()[1],
    linestyles="dashed",
    lw=1,
)

plt.hlines(
    0.5,
    xmin=plt.gca().xaxis.get_data_interval()[0],
    xmax=plt.gca().xaxis.get_data_interval()[1],
    linestyles="dashed",
    lw=1,
)

plt.vlines(
    -clf.intercept_ / clf.coef_[0],
    ymin=plt.gca().yaxis.get_data_interval()[0],
    ymax=plt.gca().yaxis.get_data_interval()[1],
    linestyles="dashed",
    lw=1,
)

plt.hlines(
    (clf.predict_proba(np.asarray(1500).reshape(-1, 1)))[0][1],
    xmin=plt.gca().xaxis.get_data_interval()[0],
    xmax=1500,
    linestyles="dashed",
    lw=1,
)

plt.vlines(
    1500,
    ymin=plt.gca().yaxis.get_data_interval()[0],
    ymax=(clf.predict_proba(np.asarray(1500).reshape(-1, 1)))[0][1],
    linestyles="dashed",
    lw=1,
)

plt.ylabel("Probability of default")
plt.xlabel("Balance")
plt.yticks([0, 0.25, 0.5, 0.75, 1.0])
plt.xlim(xmin=-100)
plt.show()

In the above example, we learnt a logistic regression model $f(x)$ with two parameters, $\beta_0$ and $\beta_1$, from the data, where
- $\beta_0 = -10.65133019 $ 
- $\beta_1 = 0.00549892 $ 

Using these two estimated parameters, we can examine the system logic of the logistic regression model to reveal its system transparency.

```{admonition} System transparency
:class: important

- When `balance`=1500 , the predicted probability of `default` is 

$$ f(1500) = \frac{1}{1 + e^{\beta_0 + \beta_1 \times 1500}} = \frac{1}{1 + e^{-10.65133019 + 0.00549892 \times 1500}} = 0.08294763. $$

- When the probability of `default` 0.5 (the commonly used threshold for binary classification), the corresponding balance can be calculated using the _log odds_ equation above:

\begin{align}
\begin{aligned}
\ln \left(\frac{\pi}{1-\pi}\right) &= \beta_0 + \beta_1 \times x \\
\ln \left(\frac{0.5}{1-0.5}\right) &= \beta_0 + \beta_1 \times x \\
x & = \frac{- \beta_0}{\beta_1} \\
x & = \frac{- (-10.65133019)}{0.00549892} \\
x & = 1936.9858426745614.
\end{aligned}
\end{align}

Therefore, to produce result `Yes`, i.e. for the probability of `default` $>$ 0.5, the `balance` should be greater than 1936.9858426745614. Likewise, to produce result `No`, i.e. the probability of `default` $<$ 0.5, the `balance` should be less than 1936.9858426745614.
```

Let us fit a logistic regression model using the [`statsmodels` API for logistic regression] next.

In [ ]:
est = logit("default2 ~ balance", df).fit()
est.summary2().tables[1]

We can see that the estimated parameters are the same as those from the `scikit-learn` API. Note the two last columns, "[0.025" and "0.975]". These are Confidence Intervals, the very concept we covered in the last chapter on Linear Regression! Now, let us fit a logistic regression model using `student` status as the input variable.

In [ ]:
est = logit("default2 ~ student", df).fit()
est.summary2().tables[1]

The coefficient associated with `student` is positive, indicating that the probability of `default` is higher for students than non-students.

## Multiple logistic regression

The logistic regression model can be extended to multiple features. A generalised logistic regression model for an instance $\mathbf{x} = [x_1, x_2, \dots, x_D]^\top$ is

$$
\ln \left( \frac{\pi}{1-\pi} \right) = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_D x_D.
$$

Let us fit a multiple logistic regression model using the `scikit-learn` API first.

Let us plot the fitted multiple logistic regression model and study the system transparency.

In [ ]:
import warnings

warnings.filterwarnings("ignore")

X_train = df.loc[:, ["balance", "income", "student2"]]
y = df.default2

clf = LogisticRegression(solver="newton-cg", penalty="none", max_iter=1000)
clf.fit(X_train, y)
print(clf)
print("classes: ", clf.classes_)
print("coefficients: ", clf.coef_)
print("intercept :", clf.intercept_)

Let us fit a multiple logistic regression model using the `statsmodels` API next.

In [ ]:
est = logit("default2 ~ balance + income + student", df).fit()
est.summary2().tables[1]

Similar parameters are estimated as using the `scikit-learn` API.

## Evaluation of logistic regression

The evaluation of logistic regression is similar to the evaluation of linear regression. The main difference is that the evaluation metrics are different. For example, the mean squared error (MSE) is not a good metric for classification. Here we introduce two common metrics for classification: accuracy and area under [receiver operating characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) (ROC) curve, i.e. AUC (or more precisely, AUROC).

### Accuracy

The accuracy is the fraction of correct predictions. It is defined as

$$
\text{accuracy} = \frac{\text{number of correct predictions}}{\text{total number of predictions}}.
$$

Let us compute the training accuracy of the multiple logistic regression model trained using `scikit-learn` API above.

In [ ]:
y_pred = clf.predict(X_train)
accuarcy = accuracy_score(y, y_pred)
print("Accuracy: ", accuarcy)

### Area under the ROC curve (AUC)

The receiver operating characteristic (ROC) curve is a plot of the true positive rate (TPR) against the false positive rate (FPR) at various threshold settings. The TPR and FPR are defined as

$$
\begin{aligned}
\text{TPR} &= \frac{\text{number of true positives}}{\text{number of positives}} \\
\text{FPR} &= \frac{\text{number of false positives}}{\text{number of negatives}}
\end{aligned}
$$

The area under the ROC curve (AUC) is a popular measure of classifier performance. The AUC is defined as the area under the ROC curve. It is a number between 0 and 1, where 0.5 is the random guess and 1 is the perfect prediction. The AUC is a good metric for evaluating the classification since it considers all possible classification thresholds.

Let us plot the ROC curve of the multiple logistic regression model trained using `scikit-learn` API above.

In [ ]:
y_prob = clf.predict_proba(X_train)[:, 1]

roc_auc = roc_auc_score(y, y_prob)
print("AUC: ", roc_auc)

clf_disp = RocCurveDisplay.from_estimator(clf, X_train, y)
plt.show()

We can see clearly the trade-off between TPR and FPR. The AUC is 0.95, indicating that the model is good.

## Exercise

**1**. Logistic regression is a **classification** algorithm.

        a. True
        
        b. False

*Compare your answer with the solution below*

```{toggle}
**a. True**
```

**2**. All the following exercises will use the [Weekly](https://github.com/pykale/transparentML/blob/main/data/Weekly.csv) dataset.

Load the dataset, convert the categories to numerical values, and fit a logistic regression model using **Volume** variable as a predictor and **Direction** variable as the response using **scikit-learn or statsmodel**. Find out the fitted model's **weight** and **bias**.

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
import pandas as pd
import numpy as np
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression


data_url = "https://github.com/pykale/transparentML/raw/main/data/Weekly.csv"
df = pd.read_csv(data_url)
df["Direction"] = df.Direction.factorize()[0]

clf = LogisticRegression(solver="newton-cg")
X_train = df.Volume.values.reshape(-1, 1)
y = df.Direction
clf.fit(X_train, y)
print("coefficients: ", clf.coef_)
print("intercept :", clf.intercept_)

**3**. Plot the **ROC curve** and find out the **training accuracy** and **AUC** of the fitted model from **Exercise 2**.

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, RocCurveDisplay

y_pred = clf.predict(X_train)
accuarcy = accuracy_score(y, y_pred)
print("Training Accuracy: ", accuarcy)

y_prob = clf.predict_proba(X_train)[:, 1]
roc_auc = roc_auc_score(y, y_prob)
print("AUC: ", roc_auc)

clf_disp = RocCurveDisplay.from_estimator(clf, X_train, y)
plt.show()

# From the ROC curve we can say that the result is almost random as it is almost close to 50%.
# The model from Exercise-2 has not learnt anything at all.

**4**. Train another logistic regression model using **Direction** as a response and all the other features as predictors. Print out all the **parameter values**.

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
X_train = df.drop(["Direction"], axis=1)

y = df.Direction

clf = LogisticRegression(solver="newton-cg")
clf.fit(X_train, y)
print("coefficients: ", clf.coef_)
print("intercept :", clf.intercept_)

**5**. Plot the **ROC curve** and find out the **training accuracy** and **AUC** metrics of the fitted model from **Exercise 4**.

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
y_pred = clf.predict(X_train)
accuarcy = accuracy_score(y, y_pred)
print("Training Accuracy: ", accuarcy)

y_prob = clf.predict_proba(X_train)[:, 1]
roc_auc = roc_auc_score(y, y_prob)
print("AUC: ", roc_auc)

clf_disp = RocCurveDisplay.from_estimator(clf, X_train, y)
plt.show()

# The model from Exercise 4 learnt very well on the training dataset.

**6**. You might notice that the model with all variables is much better than the model from **Exercise 2**. Use the skills you have learnt to perform further analysis on the data to explain why. **Hint**: Consider correlations (See section [2.2.4](https://pykale.github.io/transparentML/02-linear-reg/multi-linear-regression.html#interpreting-the-results))

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
corr_matrix = df.corr()
print(corr_matrix)
# Here from the correlation result we can see that, "Today" has a 0.72 correlation with "Direction"
# which is the main reason for getting high performance

y = df.Direction

# model with out "Today"
X_train = df.drop(["Direction", "Today"], axis=1)
clf.fit(X_train, y)
clf_disp = RocCurveDisplay.from_estimator(clf, X_train, y)
# Without today feature the model performed very bad

# model with only "Today"
X_train = df["Today"].values.reshape(-1, 1)
clf.fit(X_train, y)
clf_disp = RocCurveDisplay.from_estimator(clf, X_train, y)
# with today feature the model performed very well
# from this experiment we analyzed the important of correlation between feature and target variable